In [0]:
!pip install tensorflow==2.0.0-beta1 --upgrade
!pip install tensorflow-gpu==2.0.0-beta1 --upgrade
!pip install tfp-nightly --upgrade

!pip install git+git://github.com/andysigler/keras-mdn-layer.git@master#egg=keras-mdn-layer --upgrade

In [0]:
from __future__ import absolute_import, division, print_function

from datetime import datetime
import math
import os
import random
from shutil import copyfile

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print('Tensorflow Version: ', tf.__version__)
print('GPU: ', tf.test.gpu_device_name())
from tensorflow.keras import Sequential, layers, models, Input, optimizers

import mdn

from google.colab import drive

drive.mount('/content/gdrive')
drive_data_folder = '/content/gdrive/My Drive/Colab Notebooks/data/doodle_data'

In [0]:
drive_model_folder = os.path.join(drive_data_folder, 'models')

def dated_filename():
  return str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))


def save_model(model, name):
  blob_name = '{0}_{1}.h5'.format(name, dated_filename())
  print('Saving -> {}'.format(blob_name))
  model.save(os.path.join(drive_model_folder, blob_name))


def load_latest_model(name, custom_objects={}):
  model_files = [
      f
      for f in os.listdir(drive_model_folder)
      if name in f and 'h5' in f
  ]
  model_files.sort()
  latest_file_name = model_files[-1]
  latest_filepath = os.path.join(drive_model_folder, latest_file_name)
  print('Loading -> {}'.format(latest_filepath))
  model = tf.keras.models.load_model(
      latest_filepath, custom_objects=custom_objects)
  return model

In [0]:
def read_file(path):
  file_data = []
  with open(path, 'r') as f:
    for line in iter(f):
      line = line.strip()
      sample = line.split(',')
      file_data.append({
          'x': float(sample[1]),
          'y': float(sample[2])
      })
  return file_data


def get_cartesian(x1, y1, x2, y2):
  x_diff = x2 - x1
  y_diff = y2 - y1
  hyp = math.sqrt((x_diff * x_diff) + (y_diff * y_diff))
  rad = None
  if x_diff == 0:
    if y_diff > 0:
      rad = math.pi * 0.5
    else:
      rad = math.pi * 1.5
  else:
    rad = math.atan(y_diff / x_diff)
  return (rad, hyp)


def get_abs_radian_diff(rad_1, rad_2):
  abs_diff = abs(rad_2 - rad_1)
  if abs_diff > math.pi:
    abs_diff = (2 * math.pi) - abs_diff
  return abs_diff


def did_move_too_much(prev_rad, new_rad, hypotenuse):
  abs_diff_rad = get_abs_radian_diff(prev_rad, new_rad)
  if abs_diff_rad > (math.pi / 8):
    return True
  if (abs_diff_rad * hypotenuse) > (math.pi / 360.0):
    return True
  return False


def trim_abs_data(data):
  trimmed_data = [data[0]]
  prev_rad, _ = get_cartesian(
    trimmed_data[-1]['x'], trimmed_data[-1]['y'], data[1]['x'], data[1]['y']
  )
  for i in range(1, len(data)):
    new_rad, hypotenuse = get_cartesian(
      trimmed_data[-1]['x'], trimmed_data[-1]['y'], data[i]['x'], data[i]['y']
    )
    if did_move_too_much(prev_rad, new_rad, hypotenuse):
      trimmed_data.append(data[i - 1])
      prev_rad, _ = get_cartesian(
        trimmed_data[-1]['x'], trimmed_data[-1]['y'], data[i]['x'], data[i]['y']
      )
  trimmed_data.append(data[-1])
  return trimmed_data


def invert_abs_data(data):
  inverted_data = [
    {'x': 1.0 - d['x'], 'y': 1.0 - d['y']}
    for d in data
  ]
  flipped_data = [
    {'x': d['y'], 'y': d['x']}
    for d in data
  ]
  flipped_inverted_data = [
    {'x': 1.0 - d['x'], 'y': 1.0 - d['y']}
    for d in flipped_data
  ]
  return (data, inverted_data, flipped_data, flipped_inverted_data)


def scale_drawing(data, scale_list):
  return [
    [
      {
        l: (d[l] * scale) + ((1.0 - scale) / 2)
        for l in 'xy'
      }
      for d in data
    ]
    for scale in scale_list
  ]


def abs_data_to_delta_tensor(data):
  d_data = [
    [
      data[i][l] - data[i - 1][l]
      for l in 'xy'
    ]
    for i in range(1, len(data))
  ]
  # add the ABS XY data to the end of the sample list
  for i in range(1, len(data)):
    d_data[i - 1].append(data[i]['x'])
    d_data[i - 1].append(data[i]['y'])
  return tf.constant(
    d_data, shape=(len(d_data), len(d_data[0])), dtype='float32')


class SeqLenException(Exception):
  pass


def samples_to_sequences(t, seq_len):
  if t.shape[0] < seq_len + 1:  # one sample for the final Y value
    raise SeqLenException(
        'Length {0} too short for length {1}'.format(
            t.shape[0], seq_len))
  x_seqs = []
  y_seqs = []
  t_rel = tf.slice(t, begin=(0, 0), size=(t.shape[0], t.shape[1] - 2))
  for i in range(0, t.shape[0] - seq_len):
    new_seq = tf.slice(t, begin=[i, 0], size=[seq_len, t.shape[1]])
    rel_only = tf.slice(
      new_seq,
      begin=(0, 0),
      size=(new_seq.shape[0], new_seq.shape[1] - 2)
    )
    x = tf.reshape(new_seq, shape=(new_seq.shape[0] * new_seq.shape[1],))
    y = tf.reshape(rel_only, shape=(rel_only.shape[0] * rel_only.shape[1],))
    x_seqs.append(x)
    y_seqs.append(y)
  x_seqs = tf.stack(x_seqs, axis=0)
  y_seqs = tf.stack(y_seqs, axis=0)
  return x_seqs, y_seqs


def file_to_train_seqs(path, seq_len):
  file_data = read_file(path)
  trimmed_data = trim_abs_data(file_data)
  accum_x = []
  accum_y = []
  inverted_drawings = invert_abs_data(trimmed_data)
  scaled_drawings = []
  scale_values = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5]
#   scale_values = [0.9]
  for d in inverted_drawings:
    scaled_drawings += scale_drawing(d, scale_values)
  for d in scaled_drawings:
    drawing_tensor = abs_data_to_delta_tensor(d)
    x_seqs, y_seqs = samples_to_sequences(drawing_tensor, seq_len)
    accum_x.append(x_seqs)
    accum_y.append(y_seqs)
  return accum_x, accum_y


class RnnLenException(Exception):
  pass


def seqs_to_rnn_seqs(x, y, seq_len):
  sample_len = int(x.shape[1] / 4)
  y_start_point = seq_len * sample_len
  x_stop_point = x.shape[0] - y_start_point
  if x_stop_point <= 0:
    raise RnnLenException(
      '{0} too short for RNN seq {1} ({2} seqs)'.format(
          x.shape[0], seq_len, y_start_point))
  x_seqs = []
  for i in range(0, x_stop_point):
    this_seq = []
    for n in range(0, seq_len):
      x_sl = tf.slice(
        x, begin=(i + (n * sample_len), 0), size=(1, x.shape[1]))
      this_seq.append(x_sl)
    this_seq = tf.concat(this_seq, axis=0)
    x_seqs.append(this_seq)
  x_seqs = tf.stack(x_seqs, axis=0)
  y_seqs = tf.slice(
    y, begin=(y_start_point, 0), size=(y.shape[0] - y_start_point, y.shape[1])
  )
  assert x_seqs.shape[0] == y_seqs.shape[0], 'RNN: {0}!={1}'.format(
    x_seqs.shape[0], y_seqs.shape[0])
  return x_seqs, y_seqs


def data_to_train_seqs(data_paths, seq_len, rnn_len=None):
  accum_x_seqs = []
  accum_y_seqs = []
  accum_x_rnn = []
  accum_y_rnn = []
  for (i, path) in enumerate(data_paths):
    if i % 10 == 0:
      print('Parsing', i, '/', len(data_paths))
    try:
      x_seq_list, y_seq_list = file_to_train_seqs(path, seq_len)
      for i in range(len(x_seq_list)):
        x_seq = x_seq_list[i]
        y_seq = y_seq_list[i]
        accum_x_seqs.append(x_seq)
        accum_y_seqs.append(y_seq)
      if rnn_len:
        for i in range(len(x_seq_list)):
          x_rnn, y_rnn = seqs_to_rnn_seqs(x_seq, y_seq, rnn_len)
          accum_x_rnn.append(x_rnn)
          accum_y_rnn.append(y_rnn)
    except (SeqLenException, RnnLenException) as e:
      print('\t{0} [file: {1}]'.format(e, path.split('/')[-1]))
      continue
  x = tf.concat(accum_x_seqs, axis=0)
  y = tf.concat(accum_y_seqs, axis=0)
  x_rnn, y_rnn = (None, None)
  if rnn_len:
    x_rnn = tf.concat(accum_x_rnn, axis=0)
    y_rnn = tf.concat(accum_y_rnn, axis=0)
  return (x, y), (x_rnn, y_rnn)

In [0]:
train_sequence_len = 16  # in the end will create N * num_samples per example
train_rnn_len = 8

training_sub_models = False

doodle_csv_folder = os.path.join(drive_data_folder, 'train-data')
doodle_file_paths = [
    os.path.join(doodle_csv_folder, f)
    for f in os.listdir(doodle_csv_folder)
    if '.csv' in f
]
# doodle_file_paths = doodle_file_paths[:12]
print('Found', len(doodle_file_paths), 'CSV files')

(train_x, train_y), (train_rnn_x, train_rnn_y) = data_to_train_seqs(
  doodle_file_paths, train_sequence_len, train_rnn_len)

copy_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
if train_rnn_len:
  rnn_dataset = tf.data.Dataset.from_tensor_slices((train_rnn_x, train_rnn_y))
  # create a Y part of a dataset to copy RNN sequences
  train_rnn_y_copy = tf.reshape(
    train_rnn_x,
    shape=(
      train_rnn_x.shape[0],
      train_rnn_x.shape[1],
      int(train_rnn_x.shape[2] / 4),
      4
    )
  )
  train_rnn_y_copy = tf.slice(
    train_rnn_y_copy,
    begin=(0, 0, 0, 0),
    size=(
      train_rnn_y_copy.shape[0],
      train_rnn_y_copy.shape[1],
      train_rnn_y_copy.shape[2],
      2
    )
  )
  train_rnn_y_copy = tf.reshape(
    train_rnn_y_copy,
    shape=(
      train_rnn_y_copy.shape[0],
      train_rnn_y_copy.shape[1],
      train_rnn_y_copy.shape[2] * train_rnn_y_copy.shape[3]
    )
  )
  rnn_copy_dataset = tf.data.Dataset.from_tensor_slices((train_rnn_x, train_rnn_y_copy))

print('\n\n')
print('Train X:', train_x.shape)
print('Train Y:', train_y.shape)
if train_rnn_len:
  print('Train RNN X:', train_rnn_x.shape)
  print('Train RNN Y:', train_rnn_y.shape)
  print('Train RNN Y Copy:', train_rnn_y_copy.shape)
print('Example Sample X:')
print(train_x.numpy()[0][:8])
print('Example Sample Y:')
print(train_y.numpy()[0][:4])

In [0]:
def insert_abs_points(points, start_abs=None):
  if start_abs is None:
    start_abs = tf.constant([0, 0], dtype='float32')
  start_abs = start_abs.numpy()
  if len(start_abs) == 4:
    start_abs = [start_abs[2] - start_abs[0], start_abs[3] - start_abs[1]]
  rp = points.numpy()
  ap = [rp[0], rp[1], rp[0] + start_abs[0], rp[1] + start_abs[1]]
  for i in range(2, len(rp), 2):
    abs_x = rp[i] + ap[-2]
    abs_y = rp[i + 1] + ap[-1]
    ap.append(rp[i])
    ap.append(rp[i + 1])
    ap.append(abs_x)
    ap.append(abs_y)
  return tf.constant(ap, shape=(len(ap)), dtype='float32')


def get_sequence(seq_data, start_index):
  if len(seq_data.shape) == 2:
    ps = tf.slice(
      seq_data,
      [start_index, 0],
      [1, seq_data.shape[1]]
    )
  else:
    ps = tf.slice(
      seq_data,
      [start_index, 0, 0],
      [1, seq_data.shape[1], seq_data.shape[2]]
    )
  return tf.squeeze(ps, axis=0)


def get_random_sequence(seq_data):
  start_index = random.randint(0, seq_data.shape[0])
  return get_sequence(seq_data, start_index)


def plot_points(points, color='black'):
  points = points.numpy()
  points = [
    [points[i], points[i + 1]]
    for i in range(2, len(points), 4)
  ]
  for i in range(1, len(points)):
    plt.plot(
      [points[i - 1][0], points[i][0]],
      [points[i - 1][1], points[i][1]],
      color=color
    )


def plot_x_y(x, y, color='red'):
  plot_points(x, color='black')
  first_x_point = tf.slice(x, begin=(0,), size=(4,))
  draw_y = insert_abs_points(y, first_x_point)
  plot_points(draw_y, color=color)

    
def plot_rnn_x_y(x, y, color='red'):
  x = tf.reshape(
    x, shape=(x.shape[0] * x.shape[1],)
  )
  plot_points(x, color='black')
  last_rel_point = tf.slice(x, begin=(x.shape[0] - 4,), size=(2,))
  last_abs_point = tf.slice(x, begin=(x.shape[0] - 4,), size=(4,))
  y = tf.concat([last_rel_point, y], axis=0)
  y = insert_abs_points(y, last_abs_point)
  plot_points(y, color=color)


def draw_plot():
  plt.show()


offset = random.randint(0, train_x.shape[0])
for i in range(3):
  draw_x = get_sequence(train_x, i + offset)
  draw_y = get_sequence(train_y, i + offset)
  plot_x_y(draw_x, draw_y)
  draw_plot()

if train_rnn_len:
  for i in range(3):
    offset = random.randint(0, train_rnn_x.shape[0])
    draw_rnn_x = get_sequence(train_rnn_x, offset)
    draw_rnn_y = get_sequence(train_rnn_y, offset)
    plot_rnn_x_y(draw_rnn_x, draw_rnn_y)
    draw_plot()

In [0]:
tf.keras.backend.clear_session()

In [0]:
num_input_dims = train_sequence_len * 4
num_output_dims = train_sequence_len * 2

num_conv_layers = 3
conv_filters_by_layer = [
  32 * (2 ** n)
  for n in range(num_conv_layers)
]
conv_settings = {
  'kernel_size': (1, 5),
  'strides': (1, 2),
  'padding': 'same',
  'activation': 'tanh',
  'use_bias': True
}

num_latent_dims = int((train_sequence_len / (2 ** num_conv_layers)) * conv_filters_by_layer[-1])

num_lstm_units = 256
rnn_input_shape = (
  train_rnn_len,
  int(train_sequence_len / (2 ** num_conv_layers)),
  conv_filters_by_layer[-1]
)

num_mdn_mixes = 16
mdn_loss_func = mdn.get_mixture_loss_func(num_output_dims, num_mdn_mixes)


def create_encoder():
  input_layer = Input(shape=(num_input_dims,), dtype='float32')
  x = input_layer
  
  # CONV LAYERS
  x = layers.Reshape((1, train_sequence_len, 4))(x)  # height = 1
  for n in range(num_conv_layers):
    x = layers.Conv2D(filters=conv_filters_by_layer[n], **conv_settings)(x)

  # FLATTEN
  output_layer = layers.Flatten()(x)
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def create_rnn_encoder():
  input_layer = Input(shape=(train_rnn_len, num_input_dims), dtype='float32')
  x = input_layer
  
  # CONV LAYERS
  x = layers.Reshape((train_rnn_len, train_sequence_len, 4))(x)  # height = 8
  for n in range(num_conv_layers):
    x = layers.Conv2D(filters=conv_filters_by_layer[n], **conv_settings)(x)
  
  output_layer = layers.Reshape((train_rnn_len, x.shape[-2] * x.shape[-1]))(x)
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def create_rnn_lstm():
  input_layer = Input(shape=(train_rnn_len, num_latent_dims), dtype='float32')
  x = input_layer

  # RNN LAYERS
  x = layers.LSTM(num_lstm_units, return_sequences=True)(x)
  x = layers.LSTM(num_lstm_units, return_sequences=False)(x)
  
  # MDN LAYERS
  output_layer = layers.Dense(num_latent_dims, activation='tanh')(x)
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def create_decoder():
  input_layer = Input(shape=(num_latent_dims,), dtype='float32')
  x = input_layer
  
  # CONV LAYERS
  x = layers.Reshape((
    1,
    int(num_latent_dims / conv_filters_by_layer[-1]),
    conv_filters_by_layer[-1])
  )(x)  # height = 1
  for n in range(1, num_conv_layers):
    x = layers.Conv2DTranspose(filters=conv_filters_by_layer[::-1][n], **conv_settings)(x)
  x = layers.Conv2DTranspose(filters=2, **conv_settings)(x)

  # FLATTEN
  output_layer = layers.Flatten()(x)
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def create_rnn_decoder():
  input_layer = Input(shape=(train_rnn_len, num_latent_dims), dtype='float32')
  x = input_layer
  
  # CONV LAYERS
  x = layers.Reshape((
    train_rnn_len,
    int(num_latent_dims / conv_filters_by_layer[-1]),
    conv_filters_by_layer[-1])
  )(x)  # height = 1
  for n in range(1, num_conv_layers):
    x = layers.Conv2DTranspose(filters=conv_filters_by_layer[::-1][n], **conv_settings)(x)
  x = layers.Conv2DTranspose(filters=2, **conv_settings)(x)

  # FLATTEN
  output_layer = layers.Reshape((train_rnn_len, num_output_dims))(x)
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def create_mdn_decoder():
  input_layer = Input(shape=num_latent_dims, dtype='float32')
  
  # MDN LAYERS
  output_layer = mdn.create_mdn_layers(
      input_layer, num_output_dims, num_mdn_mixes, mu_activation='tanh')
  return tf.keras.Model(inputs=input_layer, outputs=output_layer)


def mdn_predict(model, x, **kwargs):
  while len(model.input_shape) > len(x.shape):
    x = tf.expand_dims(x, axis=0)
  y = model.predict(x)
  y = tf.squeeze(y)
  mdn_y = mdn.sample_from_output(
    y, num_output_dims, num_mdn_mixes, **kwargs)
  return tf.squeeze(tf.constant(mdn_y, shape=mdn_y.shape, dtype='float32'))


def rnn_random_input(model, x, y, mdn=False):
  rand_index = random.randint(0, x.shape[0])
  seed = get_sequence(x, rand_index)
  true_rnn_y = get_sequence(y, rand_index)
  plot_rnn_x_y(seed, true_rnn_y, color='grey')
  if mdn:
    colors = ['red', 'orange', 'green', 'blue']
    sigma_temps = [0.05, 0.005, 0.001, 0.0001]
    for c, st in zip(colors, sigma_temps):
      guess = mdn_predict(model, seed, temp=1, sigma_temp=st)
      plot_rnn_x_y(seed, guess, color=c)
  else:
    guess = model.predict(tf.expand_dims(seed, axis=0))
    guess = tf.squeeze(guess, axis=0)
    plot_rnn_x_y(seed, guess, color='red')
  draw_plot()
  

def copy_random_input(model, x, y, mdn=False):
  rand_index = random.randint(0, x.shape[0])
  seed = get_sequence(x, rand_index)
  if mdn:
    colors = ['red', 'orange', 'green']
    sigma_temps = [0.005, 0.001, 0.0001]
    for c, st in zip(colors, sigma_temps):
      guess = mdn_predict(model, seed, temp=1, sigma_temp=st)
      plot_x_y(seed, guess, color=c)
  else:
    guess = model.predict(tf.expand_dims(seed, axis=0))
    guess = tf.squeeze(guess, axis=0)
    if len(seed.shape) == 2:
      seed = tf.reshape(seed, shape=(seed.shape[0] * seed.shape[1],))
    if len(guess.shape) == 2:
      guess = tf.reshape(guess, shape=(guess.shape[0] * guess.shape[1],))
    plot_x_y(seed, guess)
  draw_plot()
  

In [0]:
model_file_name = 'doodle_AAA_{0}_{1}_{2}_seq'.format(
  '{0}', train_sequence_len, train_rnn_len
)

In [0]:
loading_latest = True

if loading_latest:
  model_encoder = load_latest_model(model_file_name.format('encoder'))
  model_decoder = load_latest_model(model_file_name.format('decoder'))
else:
  model_encoder = create_encoder()
  model_decoder = create_decoder()

print('Encoder')
model_encoder.summary()
print('\n\nDecoder')
model_decoder.summary()

model_copy = Sequential([model_encoder, model_decoder])
print('\n\n')
model_copy.summary()

In [0]:
for i in range(3):
  copy_random_input(model_copy, train_x, train_y)

In [0]:
if training_sub_models:
  model_copy.compile(
    loss='mse',
    optimizer=optimizers.Adam(
      learning_rate=0.0003
    )
  )

In [0]:
if training_sub_models:
  num_epochs = 3
  batch_size = 64

  for i in range(num_epochs):
    print()
    print('Epoch {0}/{1}:'.format(i + 1, num_epochs))
    this_dataset = copy_dataset.shuffle(train_x.shape[0]).batch(batch_size)
    model_copy.fit(this_dataset, epochs=1, verbose=1)
    for i in range(3):
      copy_random_input(model_copy, train_x, train_y)

In [0]:
if training_sub_models:
  save_model(model_encoder, model_file_name.format('encoder'))
  save_model(model_decoder, model_file_name.format('decoder'))

In [0]:
loading_latest = True

if loading_latest:
  model_rnn_encoder = load_latest_model(model_file_name.format('rnn_encoder'))
  model_rnn_decoder = load_latest_model(model_file_name.format('rnn_decoder'))
else:
  model_rnn_encoder = create_rnn_encoder()
  model_rnn_decoder = create_rnn_decoder()

print('Encoder')
model_rnn_encoder.summary()
print('\n\nDecoder')
model_rnn_decoder.summary()

model_rnn_copy = Sequential([model_rnn_encoder, model_rnn_decoder])
print('\n\n')
model_rnn_copy.summary()

In [0]:
for i in range(3):
  copy_random_input(model_rnn_copy, train_rnn_x, train_rnn_y_copy)

In [0]:
if training_sub_models:
  model_rnn_copy.compile(
    loss='mse',
    optimizer=optimizers.Adam(
      learning_rate=0.0003
    )
  )

In [0]:
if training_sub_models:
  num_epochs = 4
  batch_size = 64

  for i in range(num_epochs):
    print()
    print('Epoch {0}/{1}:'.format(i + 1, num_epochs))
    this_dataset = rnn_copy_dataset.shuffle(train_x.shape[0]).batch(batch_size)
    model_rnn_copy.fit(this_dataset, epochs=1, verbose=1)
    for i in range(3):
      copy_random_input(model_rnn_copy, train_rnn_x, train_rnn_y_copy)

In [0]:
if training_sub_models:
  save_model(model_rnn_encoder, model_file_name.format('rnn_encoder'))
  save_model(model_rnn_decoder, model_file_name.format('rnn_decoder'))

In [0]:
loading_latest = True

if loading_latest:
  model_mdn_decoder = load_latest_model(model_file_name.format('mdn_decoder'),{
    'elu_plus_one_plus_epsilon': mdn.elu_plus_one_plus_epsilon
  })
else:
  model_mdn_decoder = create_mdn_decoder()

print('MDN Decoder')
model_mdn_decoder.summary()

model_encoder.trainable = False

model_mdn_copy = Sequential([model_encoder, model_mdn_decoder])
print('\n\n')
model_mdn_copy.summary()

In [0]:
for i in range(3):
  copy_random_input(model_mdn_copy, train_x, train_y, mdn=True)

In [0]:
if training_sub_models:
  model_mdn_copy.compile(
    loss=mdn_loss_func,
    optimizer=optimizers.Adam(
      learning_rate=0.0003
    )
  )

In [0]:
if training_sub_models:
  num_epochs = 2
  batch_size = 64

  for i in range(num_epochs):
    print()
    print('Epoch {0}/{1}:'.format(i + 1, num_epochs))
    this_dataset = copy_dataset.shuffle(train_x.shape[0]).batch(batch_size)
    model_mdn_copy.fit(this_dataset, epochs=1, verbose=1)
    for i in range(3):
      copy_random_input(model_mdn_copy, train_x, train_y, mdn=True)

In [0]:
if training_sub_models:
  save_model(model_mdn_decoder, model_file_name.format('mdn_decoder'))

In [0]:
loading_latest = True

if loading_latest:
  model_rnn_lstm = load_latest_model(model_file_name.format('rnn_lstm'))
else:
  model_rnn_lstm = create_rnn_lstm()

print('RNN LSTM')
model_rnn_lstm.summary()

model_rnn_encoder.trainable = False
model_mdn_decoder.trainable = False

model_rnn_mdn = Sequential([model_rnn_encoder, model_rnn_lstm, model_mdn_decoder])
print('\n\n')
model_rnn_mdn.summary()

In [0]:
for i in range(3):
  rnn_random_input(model_rnn_mdn, train_rnn_x, train_rnn_y, mdn=True)

In [0]:
model_rnn_mdn.compile(
  loss=mdn_loss_func,
  optimizer=optimizers.Adam(
    learning_rate=0.0003
  )
)

In [0]:
num_epochs = 50
batch_size = 64

for i in range(num_epochs):
  print()
  print('Epoch {0}/{1}:'.format(i + 1, num_epochs))
  this_dataset = rnn_dataset.shuffle(train_x.shape[0]).batch(batch_size)
  model_rnn_mdn.fit(this_dataset, epochs=1, verbose=1)
  for i in range(3):
    rnn_random_input(model_rnn_mdn, train_rnn_x, train_rnn_y, mdn=True)

In [0]:
save_model(model_rnn_lstm, model_file_name.format('rnn_lstm'))

In [0]:
save_model(model_rnn_mdn, model_file_name.format('rnn_mdn'))